In [6]:
!pip install optuna

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!nvidia-smi

Thu Jan 20 13:36:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

import optuna
import gc
xgb.__version_

AttributeError: ignored

In [8]:
train = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/test.csv')
columns = [col for col in train.columns.to_list() if col not in ['id','song_popularity']]
data=train[columns]
target=train['song_popularity']

In [22]:
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import log_loss
import lightgbm as lgbm
def objective(trial):
  train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15,random_state=42)
  param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
  }
  model = lgbm.LGBMClassifier(**param_grid)
  model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=300,verbose=False)
    
  model.fit(
            train_x,
            train_y,
            eval_set=[(test_x, test_y)],
            early_stopping_rounds=300,verbose=False)
  preds = model.predict(test_x)
  roc = accuracy_score(test_y, preds)
  return roc
      

In [23]:
study = optuna.create_study(direction='maximize',study_name='song-pop_trials_200')
study.optimize(objective, n_trials=200)

[I 2022-01-20 13:44:02,582] A new study created in memory with name: song-pop_trials_200
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:548: UserWarning:

The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].

[I 2022-01-20 13:44:03,552] Trial 0 finished with value: 0.639 and parameters: {'n_estimators': 10000, 'learning_rate': 0.07582176233803521, 'num_leaves': 980, 'max_depth': 8, 'min_data_in_leaf': 8700, 'lambda_l1': 60, 'lambda_l2': 35, 'min_gain_to_split': 0.7941541874225622, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 0.639.
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:548: UserWarning:

The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].

[I 2022-01-20 13:44:04,186] Trial 1 finished with value: 0.639 and parameters: {'n_estim

In [24]:
study.best_params

{'bagging_fraction': 0.7,
 'bagging_freq': 1,
 'feature_fraction': 0.5,
 'lambda_l1': 0,
 'lambda_l2': 30,
 'learning_rate': 0.12175906487922063,
 'max_depth': 8,
 'min_data_in_leaf': 3100,
 'min_gain_to_split': 1.543021792185368,
 'n_estimators': 10000,
 'num_leaves': 2340}

In [25]:
study.best_value

0.6456666666666667